In [ ]:
### Setup Env for Local
import os; from importlib import reload;
%cd {os.environ['PROJECT_PATH']}
from utils.constants import *

In [ ]:

from dash import callback, html, dash_table, dcc, Input, Output, State, MATCH
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import jupyter_dash

from dash_app.cache import redis_store

In [ ]:
### App Setup
from dash_app.cache import redis_store
from data import load_data; reload(load_data)
from dash_app.components import datatable_comp; reload(datatable_comp)
DataTableAIO = datatable_comp.DataTableAIO

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    dbc.Button(id='button-load-data', children=['Load Data'], n_clicks=0, color="primary"),
    html.Div(id='datatable-container'),
    html.Div(id='DataTableAIO_overview'),
    # DataTableAIO(df=dfAll.head().describe().reset_index(), aio_id='my-dataset'),
    html.Div(id='graphs'),
    html.Div(id='logs')
])

# Display a DataTableIO based off of the dataset
@callback(
    Output('DataTableAIO_overview', 'children'),
    # Output(DataTableAIO.ids.store('my-dataset'), 'data'),
    # Output('my-dataset', 'df'),
    # Output({
    #     'component': 'DataTableAIO',
    #     'subcomponent': '',
    #     'aio_id': "my-dataset",
    # }, 'data'),
    Input('button-load-data', 'n_clicks'))
def display_datatable(n_clicks):
    print('display_datatable')
    # return 'asdf'
    # if n_clicks==0 : return [f'n_clicks {n_clicks}']
    # if n_clicks==0 : return pd.DataFrame().to_dict('records')

    df = load_data.load_dataframe_from_pickle('data/interim')
    df = df.drop('ts', axis=1)

    return DataTableAIO(aio_id="my-dataset", df=df.sample(50).describe().reset_index())

    df_key = redis_store.save(df)
    # return {'df':df_key}
    # return df.describe().reset_index().to_json(orient='records')
    # return {
    #     'df':df.describe().reset_index().to_dict(orient='records')
    # }

# Graph the filtered data
@callback(
    Output('graphs', 'children'),
    Input(DataTableAIO.ids.datatable('my-dataset'), 'filter_query'),
    State(DataTableAIO.ids.store('my-dataset'), 'data')
)
def update_graph(filter_query, store):
    # Fetch the data that `DataTableAIO` stored in Redis
    print("store", store)
    df = redis_store.load(store['df'])
    # Filter the data using the method that `DataTableAIO` exposes
    # and uses internally
    df = DataTableAIO.filter_df(df, filter_query)
    # Graph each column of the data
    divs = html.Div([
        html.Div([
            html.B(c),
            dcc.Graph(figure=px.histogram(x=df[c]))
        ]) for c in df.columns[:2]
    ])
    return divs


app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '600px'})

In [ ]:
# dfAll = load_data.load_dataframe_from_pickle('data/interim')

In [ ]:
from dash_app.components import datatable_comp; reload(datatable_comp)
from data import load_data; reload(load_data)


app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

TEST_TYPES = ['HLV','LLV','rec','res','bre']
app.layout = dbc.Card([
        dbc.CardBody([
            html.H4("Load VCG Data", className="card-title"),
            html.P(
                "Select which types of VCG readings to work with. (Read preprocessed VCG data from local file system.)",
                className="card-text"),
            dbc.InputGroup([
                dcc.Dropdown(id='input_test_types', multi=True,
                             options=[{'value':i,'label':i} for i in TEST_TYPES], value=TEST_TYPES[:2],
                            style={'minWidth':'400px'}),
                dbc.Button(id='button-load', children=['Load Data'], n_clicks=0, color="primary")
            ]),
        ]),
        dbc.Alert("Success! Your data loaded Successfully",id="data-load-success",dismissable=True,fade=True,duration=2000,is_open=False,),
        # html.Pre('\n'.join(['asdf','asdf2','qwer']), style={'overflow-y':'scroll', 'max-height':'100px', 'background':'grey'}),
        dcc.Store('raw-data')
])

@app.callback(
    [Output('raw-data','data'), Output('data-load-success','is_open')],
    State('input_test_types','value'), Input('button-load','n_clicks') 
)
def button_clicked(types, n):
    print('input-file-pattern, button-load', types, n)
    dfAll = load_data.load_dataframe_from_pickle('data/interim', '|'.join(types or []))
    dfAll.ts = dfAll.ts.astype(int)
    df_key = redis_store.save(dfAll)
    return df_key, True

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '600px'})

## Turn into Component

In [ ]:
import uuid
from dash import callback, html, dcc, Input, Output, State, MATCH

# from dash_app.components import datatable_comp; reload(datatable_comp)
from data import load_data; reload(load_data)

TEST_TYPES = ['HLV','LLV','rec','res','bre']

class PythonDataLoaderAIO(html.Div):
    class ids:
        dropdown = lambda aio_id: {'aio_id': aio_id,'component': 'PythonDataLoaderAIO','subcomponent': 'dropdown'}
        button = lambda aio_id: {'aio_id': aio_id,'component': 'PythonDataLoaderAIO','subcomponent': 'button'}
        store = lambda aio_id: {'aio_id': aio_id,'component': 'PythonDataLoaderAIO','subcomponent': 'store'}
        msg = lambda aio_id: {'aio_id': aio_id,'component': 'PythonDataLoaderAIO','subcomponent': 'msg'}
    ids = ids
    
    def __init__(self, title="", aio_id=None):
        if aio_id is None: aio_id = str(uuid.uuid4())
    
        super().__init__([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Load VCG Data", className="card-title"),
                    html.P(
                        "Select which types of VCG readings to work with. (Read preprocessed VCG data from local file system.)",
                        className="card-text"),
                    dbc.InputGroup([
                        dcc.Dropdown(id=self.ids.dropdown(aio_id), multi=True,
                                     options=[{'value':i,'label':i} for i in TEST_TYPES], value=TEST_TYPES[:2],
                                    style={'minWidth':'400px'}),
                        dbc.Button(id=self.ids.button(aio_id), children=['Load Data'], n_clicks=0, color="primary")
                    ]),
                    dbc.Alert(id=self.ids.msg(aio_id), children="Success! Your data loaded Successfully",dismissable=True,fade=True,duration=2000,is_open=False,),
                ]),
                dcc.Store(id=self.ids.store(aio_id))
            ])
        ])
        
    @callback(
        [Output(ids.store(MATCH),'data'), Output(ids.msg(MATCH),'is_open')],
        State(ids.dropdown(MATCH),'value'), Input(ids.button(MATCH),'n_clicks'))
    def load_data(types, n):
        print('input-file-pattern, button-load', types, n)
        if n== 0 : return '', False
        dfAll = load_data.load_dataframe_from_pickle('data/interim', '|'.join(types or []))
        dfAll.ts = dfAll.ts.astype(int)
        df_key = redis_store.save(dfAll)
        return df_key, True
        

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
comp = PythonDataLoaderAIO(aio_id='inputs')
app.layout = html.Div([
    comp,
    html.Pre("outputs loaded..", id='outputs')
])

@app.callback(Output('outputs','children'), Input(comp.ids.store('inputs'), 'data'))
def data_source_updated(df):
    print('data_source_updated', df)
    return df

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '200px'})

In [ ]:
from dash_app.components import load_data_nipams_comp
PythonDataLoaderAIO = load_data_nipams_comp.PythonDataLoaderAIO

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
comp = PythonDataLoaderAIO(aio_id='inputs')
app.layout = html.Div([
    comp,
    html.Pre("outputs loaded..", id='outputs')
])

@app.callback(Output('outputs','children'), Input(comp.ids.store('inputs'), 'data'))
def data_source_updated(df):
    print('data_source_updated', df)
    return df

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '200px'})

In [ ]:


app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
comp = PythonDataLoaderAIO(aio_id='inputs')
app.layout = html.Div([
    comp,
    html.Pre("outputs loaded..", id='outputs')
])

@app.callback(Output('outputs','children'), Input(comp.ids.store('inputs'), 'data'))
def data_source_updated(df):
    print('data_source_updated', df)
    return df

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '200px'})

# Nipams Overview Data Component

In [ ]:
from dash_app.cache import redis_store
from dash_app.components import datatable_comp; reload(datatable_comp)
DataTableAIO = datatable_comp.DataTableAIO

class NipamsDataOverviewAIO(html.Div):
    class ids:
        store = lambda aio_id: {'aio_id': aio_id,'component': 'NipamsDataOverviewAIO','subcomponent': 'store'}
        table = lambda aio_id: {'aio_id': aio_id,'component': 'NipamsDataOverviewAIO','subcomponent': 'table'}
        graphs = lambda aio_id: {'aio_id': aio_id,'component': 'NipamsDataOverviewAIO','subcomponent': 'graphs'}
        
    ids = ids
    
    def __init__(self, aio_id=None):
        if aio_id is None: aio_id = str(uuid.uuid4())
    
        super().__init__([
            dbc.Card([
                dbc.CardBody([
                    html.H4("Overview VCG Data", className="card-title"),
                    html.P(
                        "Filter Dataset & View Resulting Summaries",
                        className="card-text"),
                    html.Div(id=self.ids.table(aio_id)),
                    html.Div(id=self.ids.graphs(aio_id))
                ]),
                dcc.Store(id=self.ids.store(aio_id))
            ])
        ])
        

        
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
input_data_comp = PythonDataLoaderAIO(aio_id='input_data_comp')
# explore_data_comp = NipamsDataOverviewAIO(aio_id='explore_data_comp')
app.layout = html.Div([
    input_data_comp,
    # explore_data_comp,
    html.Pre("outputs loaded..", id='outputs')])

# @app.callback(Output('outputs','children'), Input(input_data_comp.ids.store('input_data_comp'), 'data'))
# def data_source_updated(df):print('data_source_updated', df);return df

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  


In [ ]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
input_data_comp = PythonDataLoaderAIO(aio_id='inputs')
explore_data_comp = NipamsDataOverviewAIO(aio_id='explore_data_comp')
app.layout = html.Div([
    input_data_comp,
    explore_data_comp,
    html.Pre("outputs loaded..", id='outputs')])

# @app.callback(Output('outputs','children'), Input(input_data_comp.ids.store('input_data_comp'), 'data'))
# def data_source_updated(df):print('data_source_updated', df);return df

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  


In [ ]:
from dash_app.components import load_data_nipams_comp
PythonDataLoaderAIO = load_data_nipams_comp.PythonDataLoaderAIO

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
comp = PythonDataLoaderAIO(aio_id='inputs')
app.layout = html.Div([
    comp,
    html.Pre("outputs loaded..", id='outputs')
])

@app.callback(Output('outputs','children'), Input(comp.ids.store('inputs'), 'data'))
def data_source_updated(df):
    print('data_source_updated', df)
    return df

app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '200px'})

In [ ]:
import dash_app.cache as cache
reload(cache)